In [2]:
import warnings
warnings.simplefilter('ignore')

import os
import re
import gc
import glob
from collections import Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm
tqdm.pandas()

import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk import ne_chunk, pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import spacy
from textblob import TextBlob
import gensim

STOP_WORDS = set(stopwords.words('english'))
SPACY_NER_MODEL = spacy.load('en_core_web_sm')

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=4, progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
%%time

train = pd.read_pickle('./data/train_data.pkl')
valid = pd.read_pickle('./data/valid_data.pkl')

# 暂时不用 title
train.drop(['title'], axis=1, inplace=True)
valid.drop(['title'], axis=1, inplace=True)
gc.collect()

display(train)
display(valid)

,prompt_answer,target,candidate,id,text,recall_rank
0,What is the formula for calculating the nullit...,1,17458663,3,The nullity of a graph in the mathematical sub...,1
1,What is the formula for calculating the nullit...,0,17458495,3,"In graph theory, a branch of mathematics, the ...",2
2,What is the formula for calculating the nullit...,0,745714,3,"In the mathematical theory of matroids, a grap...",3
3,What is the formula for calculating the nullit...,0,37123010,3,"In the mathematical theory of matroids, the ra...",4
4,What is the formula for calculating the nullit...,0,244321,3,"In combinatorics, a branch of mathematics, a m...",5
...,...,...,...,...,...,...
29999995,What is the Ukrainian Physicists' Tournament f...,0,68001314,68891,Andrey Valeryevich Andreychenko (; born 29 Mar...,996
29999996,What is the Ukrainian Physicists' Tournament f...,0,52096152,68891,The 2016 European championships of internation...,997
29999997,What is the Ukrainian Physicists' Tournament f...,0,53113196,68891,Curling at the 2017 Winter Universiade was hel...,998
29999998,What is the Ukrainian Physicists' Tournament f...,0,57749759,68891,The 2018–19 Ukrainian Football Amateur League ...,999


,prompt_answer,target,candidate,id,text,recall_rank
0,What is the definition of a solvent? A substan...,0,37431,0,"thumb|Ethyl acetate, nail polish solvent. A so...",1
1,What is the definition of a solvent? A substan...,0,2740944,0,"Universal solvent may refer to: *Water, descri...",2
2,What is the definition of a solvent? A substan...,1,73305281,0,A solvent is a substance that is liquid at the...,3
3,What is the definition of a solvent? A substan...,0,5048079,0,A solvent dye is a dye soluble in organic solv...,4
4,What is the definition of a solvent? A substan...,0,229643,0,Molality is a measure of the number of moles o...,5
...,...,...,...,...,...,...
2526995,"Where is the Komatsu 960E-1 assembled? Peoria,...",0,25121593,2526,"KOA Corporation (Japanese: コーア株式会社, Kōa kabush...",996
2526996,"Where is the Komatsu 960E-1 assembled? Peoria,...",0,15040535,2526,The Mitsubishi Colt (A20) was one of their fir...,997
2526997,"Where is the Komatsu 960E-1 assembled? Peoria,...",0,62697879,2526,The KTM 250 FRR was a racing motorcycle made b...,998
2526998,"Where is the Komatsu 960E-1 assembled? Peoria,...",0,1054261,2526,The Mitsubishi 6A1 engine is a series of pisto...,999


CPU times: user 29.5 s, sys: 13.7 s, total: 43.1 s
Wall time: 43.1 s


In [4]:
# train 负采样

train_pos = train[train['target'] == 1].copy().reset_index(drop=True)
train_neg = train[train['target'] == 0].copy().reset_index(drop=True)
train_neg = train_neg.sample(n=300000, random_state=42).copy().reset_index(drop=True)
train = pd.concat([train_pos, train_neg]).sort_values(
    ['id', 'recall_rank'], ascending=[True, True]).reset_index(drop=True)
print(train['target'].mean())

0.0908842968444594


In [5]:
display(train)

,prompt_answer,target,candidate,id,text,recall_rank
0,What is the formula for calculating the nullit...,1,17458663,3,The nullity of a graph in the mathematical sub...,1
1,What is the formula for calculating the nullit...,0,2472880,3,"In mathematics, a biased graph is a graph with...",68
2,What is the formula for calculating the nullit...,0,59747277,3,thumb|An illustration of the three sets in the...,286
3,What is the formula for calculating the nullit...,0,37868330,3,"In probability theory, a transition rate matri...",292
4,What is the formula for calculating the nullit...,0,48413286,3,"In discrete mathematics, the Bregman–Minc ineq...",344
...,...,...,...,...,...,...
329986,What is the Ukrainian Physicists' Tournament f...,0,49478061,68891,The Ukrainian men's national 3x3 team represen...,394
329987,What is the Ukrainian Physicists' Tournament f...,0,26757624,68891,The 2009–10 Ukrainian League Cup was the only ...,430
329988,What is the Ukrainian Physicists' Tournament f...,0,67123178,68891,This is a comprehensive list of victories of t...,554
329989,What is the Ukrainian Physicists' Tournament f...,0,73925214,68891,FC Shakhta Ukraina Ukrainsk was a football tea...,610


In [9]:
# 输入 datafram 输出特征

def make_features(dataframe):
    
    dataframe['token1'] = dataframe['prompt_answer'].parallel_apply(lambda x: x.split()) 
    dataframe['token2'] = dataframe['text'].parallel_apply(lambda x: x.split())
    
    dataframe.drop(['prompt_answer', 'text'], axis=1, inplace=True)
    gc.collect()
    
    def word_overlap_count(token1, token2): 
        def check_is_stop_word(word):
            return word not in STOP_WORDS
        words1 = list(filter(check_is_stop_word, token1))
        words2 = list(filter(check_is_stop_word, token2))
        return len(set(words1).intersection(set(words2)))
    
    def ngrams(token, n):
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]
    
    def ngram_co_occurrence(token1, token2, n):
        ngrams1 = set(ngrams(token1, n))
        ngrams2 = set(ngrams(token2, n))
        common_ngrams = ngrams1.intersection(ngrams2)
        return len(common_ngrams)
    
    dataframe['words_overlap_count'] = dataframe.parallel_apply(
        lambda row: word_overlap_count(row['token1'], row['token2']), axis=1)
    
    dataframe['2gram_overlap_count'] = dataframe.parallel_apply(
        lambda row: ngram_co_occurrence(row['token1'], row['token2'], 2), axis=1)

    dataframe['3gram_overlap_count'] = dataframe.parallel_apply(
        lambda row: ngram_co_occurrence(row['token1'], row['token2'], 3), axis=1)

    dataframe['4gram_overlap_count'] = dataframe.parallel_apply(
        lambda row: ngram_co_occurrence(row['token1'], row['token2'], 4), axis=1)
    
    dataframe['text_token_length'] = dataframe['token2'].apply(len)
    
    dataframe['words_overlap_ratio'] = dataframe['words_overlap_count'] / (dataframe['text_token_length'] + 1)
    dataframe['2gram_overlap_ratio'] = dataframe['2gram_overlap_count'] / (dataframe['text_token_length'] + 1)
    dataframe['3gram_overlap_ratio'] = dataframe['3gram_overlap_count'] / (dataframe['text_token_length'] + 1)
    dataframe['4gram_overlap_ratio'] = dataframe['4gram_overlap_count'] / (dataframe['text_token_length'] + 1)
    
    dataframe.drop(['token1', 'token2'], axis=1, inplace=True)
    gc.collect()
    
    return dataframe

In [10]:
%%time

bs = 100000
n_splits = len(train) // bs if len(train) % bs == 0 else len(train) // bs + 1

for i in range(n_splits):
    print(f'{i*bs} start')

    if bs*(i+1) < len(train):
        dataframe = train[bs*i:bs*(i+1)].copy().reset_index(drop=True)
    else:
        dataframe = train[bs*i:].copy().reset_index(drop=True)
        
    dataframe = make_features(dataframe)
    display(dataframe)
    dataframe.to_pickle(f'train_features_{i}.pkl')
    del dataframe; gc.collect()

0 start


,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,1,17458663,3,1,11,16,11,6,330,0.033233,0.048338,0.033233,0.018127
1,0,2472880,3,68,9,10,4,1,1534,0.005863,0.006515,0.002606,0.000651
2,0,59747277,3,286,5,4,1,0,975,0.005123,0.004098,0.001025,0.000000
3,0,37868330,3,292,3,3,1,0,279,0.010714,0.010714,0.003571,0.000000
4,0,48413286,3,344,5,5,2,0,589,0.008475,0.008475,0.003390,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,35184346,20678,690,0,0,0,0,1185,0.000000,0.000000,0.000000,0.000000
99996,0,25233839,20678,749,0,0,0,0,64,0.000000,0.000000,0.000000,0.000000
99997,0,72067116,20678,778,0,0,0,0,108,0.000000,0.000000,0.000000,0.000000
99998,0,27057753,20678,875,0,0,0,0,219,0.000000,0.000000,0.000000,0.000000


100000 start


,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,0,58611075,20678,926,3,0,0,0,354,0.008451,0.000000,0.000000,0.0
1,1,4485403,20681,1,7,6,2,0,3756,0.001863,0.001597,0.000532,0.0
2,0,5302845,20681,224,1,1,0,0,1033,0.000967,0.000967,0.000000,0.0
3,0,46863916,20681,225,3,2,0,0,326,0.009174,0.006116,0.000000,0.0
4,0,423832,20681,497,2,2,0,0,359,0.005556,0.005556,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,70788340,41754,606,1,1,0,0,406,0.002457,0.002457,0.000000,0.0
99996,0,43072020,41754,624,1,2,0,0,381,0.002618,0.005236,0.000000,0.0
99997,0,57442012,41754,644,0,2,0,0,3362,0.000000,0.000595,0.000000,0.0
99998,0,651485,41754,701,6,4,1,0,1720,0.003486,0.002324,0.000581,0.0


200000 start


,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,0,27400699,41754,998,0,1,0,0,202,0.000000,0.004926,0.000000,0.000000
1,1,19874353,41757,1,21,20,7,4,1371,0.015306,0.014577,0.005102,0.002915
2,0,23305394,41757,79,2,0,0,0,61,0.032258,0.000000,0.000000,0.000000
3,0,1884226,41757,179,3,2,0,0,318,0.009404,0.006270,0.000000,0.000000
4,0,1528995,41757,275,1,0,0,0,40,0.024390,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,2175813,62695,698,9,3,0,0,2254,0.003991,0.001330,0.000000,0.000000
99996,0,63345354,62695,716,1,0,0,0,155,0.006410,0.000000,0.000000,0.000000
99997,0,32961823,62695,855,1,0,0,0,81,0.012195,0.000000,0.000000,0.000000
99998,0,21152191,62695,874,3,0,0,0,625,0.004792,0.000000,0.000000,0.000000


300000 start


,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,0,32566,62698,200,4,3,0,0,1377,0.002903,0.002177,0.0,0.0
1,0,21227565,62698,248,9,4,0,0,2281,0.003944,0.001753,0.0,0.0
2,0,70470456,62698,266,1,0,0,0,17,0.055556,0.000000,0.0,0.0
3,0,13936369,62698,363,3,2,0,0,552,0.005425,0.003617,0.0,0.0
4,0,4320615,62698,390,4,4,0,0,2457,0.001627,0.001627,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29986,0,49478061,68891,394,3,1,0,0,153,0.019481,0.006494,0.0,0.0
29987,0,26757624,68891,430,5,1,0,0,1817,0.002750,0.000550,0.0,0.0
29988,0,67123178,68891,554,0,0,0,0,887,0.000000,0.000000,0.0,0.0
29989,0,73925214,68891,610,2,1,0,0,146,0.013605,0.006803,0.0,0.0


CPU times: user 5min 11s, sys: 1min 34s, total: 6min 45s
Wall time: 9min 49s


In [11]:
%%time

bs = 100000
n_splits = len(valid) // bs if len(valid) % bs == 0 else len(valid) // bs + 1

for i in range(n_splits):
    print(f'{i*bs} start')
    
    if bs*(i+1) < len(valid):
        dataframe = valid[bs*i:bs*(i+1)].copy().reset_index(drop=True)
    else:
        dataframe = valid[bs*i:].copy().reset_index(drop=True)
        
    dataframe = make_features(dataframe)
    display(dataframe)
    dataframe.to_pickle(f'valid_features_{i}.pkl')
    del dataframe; gc.collect()

0 start


,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,0,37431,0,1,4,3,0,0,3710,0.001078,0.000808,0.000000,0.000000
1,0,2740944,0,2,3,0,0,0,36,0.081081,0.000000,0.000000,0.000000
2,1,73305281,0,3,8,12,7,6,4104,0.001949,0.002923,0.001705,0.001462
3,0,5048079,0,4,1,0,0,0,167,0.005952,0.000000,0.000000,0.000000
4,0,229643,0,5,5,4,1,0,2334,0.002141,0.001713,0.000428,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,58055739,99,996,0,0,0,0,505,0.000000,0.000000,0.000000,0.000000
99996,0,38776215,99,997,0,0,0,0,458,0.000000,0.000000,0.000000,0.000000
99997,0,43344479,99,998,4,2,1,0,605,0.006601,0.003300,0.001650,0.000000
99998,0,25025517,99,999,4,2,0,0,3397,0.001177,0.000589,0.000000,0.000000


100000 start


,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,1,674969,100,1,7,6,4,3,880,0.007946,0.006810,0.004540,0.003405
1,0,43551551,100,2,2,0,0,0,207,0.009615,0.000000,0.000000,0.000000
2,0,27127825,100,3,0,0,0,0,59,0.000000,0.000000,0.000000,0.000000
3,0,65962788,100,4,1,0,0,0,169,0.005882,0.000000,0.000000,0.000000
4,0,6521538,100,5,0,1,0,0,133,0.000000,0.007463,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,47190687,199,996,5,2,0,0,344,0.014493,0.005797,0.000000,0.000000
99996,0,28382142,199,997,11,9,5,3,16155,0.000681,0.000557,0.000309,0.000186
99997,0,1475520,199,998,6,2,0,0,796,0.007528,0.002509,0.000000,0.000000
99998,0,72201050,199,999,7,6,2,1,931,0.007511,0.006438,0.002146,0.001073


200000 start


...

2400000 start


,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,1,1507174,2400,1,4,0,0,0,1668,0.002397,0.000000,0.000000,0.0
1,0,17580436,2400,2,0,0,0,0,291,0.000000,0.000000,0.000000,0.0
2,0,52086631,2400,3,1,0,0,0,315,0.003165,0.000000,0.000000,0.0
3,0,32446613,2400,4,1,0,0,0,268,0.003717,0.000000,0.000000,0.0
4,0,13554259,2400,5,0,0,0,0,257,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,215539,2499,996,9,6,1,0,4951,0.001817,0.001212,0.000202,0.0
99996,0,4713918,2499,997,1,1,0,0,218,0.004566,0.004566,0.000000,0.0
99997,0,46182856,2499,998,0,0,0,0,102,0.000000,0.000000,0.000000,0.0
99998,0,10019499,2499,999,2,0,0,0,992,0.002014,0.000000,0.000000,0.0


2500000 start


,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,1,490387,2500,1,2,1,0,0,621,0.003215,0.001608,0.0,0.0
1,0,38298699,2500,2,0,0,0,0,46,0.000000,0.000000,0.0,0.0
2,0,65206136,2500,3,0,1,0,0,432,0.000000,0.002309,0.0,0.0
3,0,45344496,2500,4,0,0,0,0,34,0.000000,0.000000,0.0,0.0
4,0,22130933,2500,5,0,0,0,0,114,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26995,0,25121593,2526,996,0,0,0,0,70,0.000000,0.000000,0.0,0.0
26996,0,15040535,2526,997,0,0,0,0,1367,0.000000,0.000000,0.0,0.0
26997,0,62697879,2526,998,0,0,0,0,1037,0.000000,0.000000,0.0,0.0
26998,0,1054261,2526,999,1,0,0,0,531,0.001880,0.000000,0.0,0.0


CPU times: user 35min 24s, sys: 9min 58s, total: 45min 23s
Wall time: 1h 7min 25s


In [12]:
train = []
for filename in glob.glob('train_features_*.pkl'):
    train.append(pd.read_pickle(filename))
train = pd.concat(train).sort_values('id').reset_index(drop=True)
display(train)

,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,1,17458663,3,1,11,16,11,6,330,0.033233,0.048338,0.033233,0.018127
1,0,3955446,3,852,1,2,0,0,507,0.001969,0.003937,0.000000,0.000000
2,0,67059821,3,670,5,4,0,0,322,0.015480,0.012384,0.000000,0.000000
3,0,18412684,3,655,2,2,1,0,134,0.014815,0.014815,0.007407,0.000000
4,0,20249418,3,653,4,2,0,0,364,0.010959,0.005479,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
329986,0,49478061,68891,394,3,1,0,0,153,0.019481,0.006494,0.000000,0.000000
329987,0,67123178,68891,554,0,0,0,0,887,0.000000,0.000000,0.000000,0.000000
329988,0,73925214,68891,610,2,1,0,0,146,0.013605,0.006803,0.000000,0.000000
329989,0,45228346,68891,856,1,0,0,0,503,0.001984,0.000000,0.000000,0.000000


In [13]:
valid = []
for filename in glob.glob('valid_features_*.pkl'):
    valid.append(pd.read_pickle(filename))
valid = pd.concat(valid).sort_values('id').reset_index(drop=True)
display(valid)

,target,candidate,id,recall_rank,words_overlap_count,2gram_overlap_count,3gram_overlap_count,4gram_overlap_count,text_token_length,words_overlap_ratio,2gram_overlap_ratio,3gram_overlap_ratio,4gram_overlap_ratio
0,0,43046101,0,999,0,0,0,0,75,0.000000,0.000000,0.0,0.0
1,0,529609,0,658,2,4,0,0,1522,0.001313,0.002626,0.0,0.0
2,0,5639039,0,659,0,0,0,0,88,0.000000,0.000000,0.0,0.0
3,0,462409,0,660,0,0,0,0,21,0.000000,0.000000,0.0,0.0
4,0,3080690,0,661,1,1,0,0,66,0.014925,0.014925,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526995,0,17573213,2526,340,0,0,0,0,224,0.000000,0.000000,0.0,0.0
2526996,0,16701385,2526,341,2,0,0,0,654,0.003053,0.000000,0.0,0.0
2526997,0,63269809,2526,342,1,1,0,0,493,0.002024,0.002024,0.0,0.0
2526998,0,6000455,2526,329,1,1,0,0,2629,0.000380,0.000380,0.0,0.0


In [14]:
%%time

train.to_pickle('./data/train.pickle')
valid.to_pickle('./data/valid.pickle')

CPU times: user 94 ms, sys: 103 ms, total: 197 ms
Wall time: 206 ms
